In [1]:
from packages import *
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.data.experimental import AUTOTUNE
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import f1_score, r2_score
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize
import re
from collections import Counter

import numpy as np
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import GlobalAveragePooling1D
from tensorflow.keras.datasets import imdb

%load_ext autoreload
%autoreload 2

In [2]:
tf.__version__

'2.0.0-alpha0'

In [3]:
epsilon = 1e-7
fp = 'float32'
tf.config.gpu.set_per_process_memory_growth(True)
tf.config.gpu.set_per_process_memory_fraction(.2)
tf.keras.backend.set_floatx(fp)
tf.keras.backend.set_epsilon(epsilon)
tf.keras.backend.set_image_data_format('channels_last')

In [4]:
batch_size = 100
glove_dim = 50
input_shape = (300, glove_dim, 1)
max_length = 300
shuffle_buffer_size = batch_size*4
prefetch_buffer_size = 1
random_seed = np.random.randint(0, 100)
test_ratio = 0.33

In [5]:
def shuffle_sample(dictionary, n_sample=None, random_seed=42):
    lens = [len(l) for l in dictionary.values()]
    assert min(lens) == max(lens)
    n_data = lens[0]
    processed = {}
    for key, array in dictionary.items():
        if n_sample is not None:
            processed[key] = shuffle(array, random_state=random_seed)[:n_sample]
        else:
            processed[key] = shuffle(array, random_state=random_seed)[:n_sample]
    return processed

In [6]:
data = shuffle_sample(load_data({'review': ['text', 'stars']})['review'],
                             random_seed=random_seed
                            )
x_train, x_test, y_train, y_test = train_test_split(data['text'],
                                                    data['stars'],
                                                    test_size = test_ratio,
                                                   )

In [7]:
glove_lookup = load_pickle(os.path.join(GLOVE_DIR, 'glove-{}D.pkl'.format(glove_dim)))
unk_key = '<UNK>'

In [8]:
unk_vector = np.mean(np.array(list(glove_lookup.values())), axis=0)
glove_lookup[unk_key] = unk_vector

In [9]:
vocab_size = len(glove_lookup.keys())

In [10]:
print('Loading data...')
# (x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
x_train_tf = tf.data.Dataset.from_tensor_slices(x_train)
y_train_tf = tf.data.Dataset.from_tensor_slices(y_train - 1) # to make stars 0-indexed

x_test_tf = tf.data.Dataset.from_tensor_slices(x_test)
y_test_tf = tf.data.Dataset.from_tensor_slices(y_test - 1)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')
print('Average train sequence length: {}'.format(
    np.mean(list(map(len, x_train)), dtype=int)))
print('Average test sequence length: {}'.format(
    np.mean(list(map(len, x_test)), dtype=int)))

Loading data...
4479553 train sequences
2206347 test sequences
Average train sequence length: 602
Average test sequence length: 602


In [11]:
def clean_token(t):
    t = t.lower()
    m = re.match('^[^\w\'](\w+).*', t)
    if m is not None:
        return m.group(1)
    else:
        return t

def tokenize(s):
    tokens = []
    for t in word_tokenize(s):
        tokens.append(clean_token(t))
    return tokens

# Returns an np.array of glove embeddings for each
# word in the given string of shape (word_count, glove_dims)
def get_glove_embeddings(tokens):
    embeddings = []
    
    for i, word in enumerate(tokens):
#         if i > 
        if word in glove_lookup:
            embeddings.append(glove_lookup[word])
        else:
            embeddings.append(np.zeros(glove_dim))
            
#     return(np.array(embeddings, dtype=np.float16))
    return(np.array(embeddings, dtype=np.float32))

def test_embed(arr):
    return get_glove_embeddings(tokenize(arr))

def embed(tensor):
    return get_glove_embeddings(tokenize(str(tensor.numpy())))

@tf.function
def fix_dimensions(tensor):
    return tf.reshape(tf.image.resize_image_with_crop_or_pad(tf.expand_dims(tensor, axis=-1), 300, glove_dim), (300,50))

In [12]:
train_dataset_embed = tf.data.Dataset.map(x_train_tf,  lambda review: tf.py_function( embed, [review], tf.float32 ), num_parallel_calls=AUTOTUNE) 
test_dataset_embed = tf.data.Dataset.map(x_test_tf,  lambda review: tf.py_function( embed, [review], tf.float32 ),num_parallel_calls=AUTOTUNE) 

In [13]:
train_dataset_embed = tf.data.Dataset.map(train_dataset_embed, fix_dimensions, num_parallel_calls=AUTOTUNE) 
test_dataset_embed = tf.data.Dataset.map(test_dataset_embed, fix_dimensions, num_parallel_calls=AUTOTUNE) 

In [14]:
train_dataset = tf.data.Dataset.zip((train_dataset_embed, y_train_tf))
test_dataset = tf.data.Dataset.zip((test_dataset_embed, y_test_tf))

In [15]:
train_dataset = tf.data.Dataset.shuffle(train_dataset, buffer_size=shuffle_buffer_size)
test_dataset = tf.data.Dataset.shuffle(test_dataset, buffer_size=shuffle_buffer_size)

In [16]:
train_dataset = tf.data.Dataset.batch(train_dataset, batch_size=batch_size)
test_dataset = tf.data.Dataset.batch(test_dataset, batch_size=batch_size)

In [17]:
train_dataset = tf.data.Dataset.prefetch(train_dataset, buffer_size=prefetch_buffer_size)
test_dataset = tf.data.Dataset.prefetch(test_dataset, buffer_size=prefetch_buffer_size)

# Prep Data

In [18]:
# Step 1: Get data from tensor
X = []
Y = []
for x_batch, y_batch in train_dataset.take(10000 // batch_size):
    X.append(x_batch)
    Y.append(y_batch)

In [19]:
embed_dim = 128
lstm_out = 200
input_len = 300

In [27]:
class YelpLSTM:
    def __init__(self, hidden_size=5, output_dim=5):
        self.hidden_size = hidden_size
        self.output_dim = output_dim
    
    def build(self):
        inp = Input(shape=(input_len, glove_dim), name="input")
        #x = Embedding(2500, embed_dim, input_length = input_len)(inp)
        #x = Dropout(0.2)(x)
        x = LSTM(self.hidden_size)(inp)
        out = Dense(self.output_dim, activation="softmax")(x)
        return tf.keras.models.Model(inputs=inp, outputs=out)
        
    @tf.function
    def loss_fn(truth, logits):
        loss = tf.keras.losses.sparse_categorical_crossentropy(truth, logits)
        return loss

In [28]:
model = YelpLSTM(hidden_size=10)
model = model.build()
model.compile(optimizer="adam", metrics=['accuracy'] ,loss=YelpLSTM.loss_fn)
model.fit(train_dataset.take(200),
          epochs=5,
          #validation_split=.2,
          validation_data=test_dataset.take(200), validation_freq=1
         )

W0605 23:04:54.084853 140203855951680 tf_logging.py:161] <tensorflow.python.keras.layers.recurrent.UnifiedLSTM object at 0x7f82d46a4ac8>: Note that this layer is not optimized for performance. Please use tf.keras.layers.CuDNNLSTM for better performance on GPU.


Epoch 1/5
200/200 [==============================] - 132s 659ms/step - loss: 1.4492 - accuracy: 0.0301 - val_loss: 1.3891 - val_accuracy: 0.0351
Epoch 2/5
200/200 [==============================] - 131s 655ms/step - loss: 1.3631 - accuracy: 0.0490 - val_loss: 1.3266 - val_accuracy: 0.0635
Epoch 3/5
200/200 [==============================] - 131s 654ms/step - loss: 1.2979 - accuracy: 0.2613 - val_loss: 1.2613 - val_accuracy: 0.3629
Epoch 4/5
200/200 [==============================] - 131s 656ms/step - loss: 1.2674 - accuracy: 0.3948 - val_loss: 1.2364 - val_accuracy: 0.3607
Epoch 5/5
200/200 [==============================] - 131s 655ms/step - loss: 1.2422 - accuracy: 0.3848 - val_loss: 1.2198 - val_accuracy: 0.3327


In [29]:
model = YelpLSTM(hidden_size=20)
model = model.build()
model.compile(optimizer="adam", metrics=['accuracy'] ,loss=YelpLSTM.loss_fn)
model.fit(train_dataset.take(200),
          epochs=5,
          #validation_split=.2,
          validation_data=test_dataset.take(200), validation_freq=1
         )

W0605 23:15:50.630335 140203855951680 tf_logging.py:161] <tensorflow.python.keras.layers.recurrent.UnifiedLSTM object at 0x7f82d42d95f8>: Note that this layer is not optimized for performance. Please use tf.keras.layers.CuDNNLSTM for better performance on GPU.


Epoch 1/5
200/200 [==============================] - 132s 658ms/step - loss: 1.4473 - accuracy: 0.0510 - val_loss: 1.3940 - val_accuracy: 0.0432
Epoch 2/5
200/200 [==============================] - 131s 655ms/step - loss: 1.3280 - accuracy: 0.2346 - val_loss: 1.2935 - val_accuracy: 0.3620
Epoch 3/5
200/200 [==============================] - 131s 655ms/step - loss: 1.2814 - accuracy: 0.3286 - val_loss: 1.2378 - val_accuracy: 0.3742
Epoch 4/5
200/200 [==============================] - 131s 655ms/step - loss: 1.1899 - accuracy: 0.2669 - val_loss: 1.1775 - val_accuracy: 0.2939
Epoch 5/5
200/200 [==============================] - 131s 656ms/step - loss: 1.1510 - accuracy: 0.2436 - val_loss: 1.1536 - val_accuracy: 0.2692


In [ ]:
model = YelpLSTM(hidden_size=50)
model = model.build()
model.compile(optimizer="adam", metrics=['accuracy'] ,loss=YelpLSTM.loss_fn)
model.fit(train_dataset.take(200),
          epochs=5,
          #validation_split=.2,
          validation_data=test_dataset.take(200), validation_freq=1
         )

W0605 23:26:47.036465 140203855951680 tf_logging.py:161] <tensorflow.python.keras.layers.recurrent.UnifiedLSTM object at 0x7f82d42b8630>: Note that this layer is not optimized for performance. Please use tf.keras.layers.CuDNNLSTM for better performance on GPU.


Epoch 1/5
200/200 [==============================] - 132s 660ms/step - loss: 1.4282 - accuracy: 0.0435 - val_loss: 1.3925 - val_accuracy: 0.0507
Epoch 2/5
199/200 [============================>.] - ETA: 0s - loss: 1.4090 - accuracy: 0.0227

In [ ]:
class YelpLSTM_2:
    def __init__(self, hidden_size=20, dense_layer_dim=100, output_dim=5):
        self.hidden_size = hidden_size
        self.output_dim = output_dim
        self.dense_layer_dim = dense_layer_dim
    
    def build(self):
        inp = Input(shape=(input_len, glove_dim), name="input")
        #x = Embedding(2500, embed_dim, input_length = input_len)(inp)
        x = LSTM(self.hidden_size)(inp)
        x = Dropout(0.2)(x)
        x = Dense(dense_layer_dim, activation="relu")(x)
        x = Dropout(0.2)(x)
        out = Dense(self.output_dim, activation="softmax")(x)
        return tf.keras.models.Model(inputs=inp, outputs=out)
        
    @tf.function
    def loss_fn(truth, logits):
        loss = tf.keras.losses.sparse_categorical_crossentropy(truth, logits)
        return loss

In [ ]:
model = YelpLSTM(hidden_size=20, dense_layer)
model = model.build()
model.compile(optimizer="adam", metrics=['accuracy'] ,loss=YelpLSTM.loss_fn)
model.fit(train_dataset.take(200),
          epochs=5,
          #validation_split=.2,
          validation_data=test_dataset.take(200), validation_freq=1
         )

# Extra Feature Stuff

In [12]:
import spacy
from spacy import displacy

In [13]:
nlp = spacy.load('en')

In [23]:
i=0

In [24]:
doc = nlp(text[i])

In [25]:
for token in doc:
    print("{0}/{1} <--{2}-- {3}/{4}".format(
        token.text, token.tag_, token.dep_, token.head.text, token.head.tag_))

Total/JJ <--amod-- bill/NN
bill/NN <--ROOT-- bill/NN
for/IN <--prep-- bill/NN
this/DT <--det-- service/NN
horrible/JJ <--amod-- service/NN
service/NN <--pobj-- for/IN
?/. <--punct-- bill/NN
Over/IN <--ROOT-- Over/IN
$/$ <--nummod-- 8Gs/CD
8Gs/CD <--pobj-- Over/IN
./. <--punct-- Over/IN
These/DT <--det-- crooks/NNS
crooks/NNS <--nsubj-- had/VBD
actually/RB <--advmod-- had/VBD
had/VBD <--ROOT-- had/VBD
the/DT <--det-- nerve/NN
nerve/NN <--dobj-- had/VBD
to/TO <--aux-- charge/VB
charge/VB <--acl-- nerve/NN
us/PRP <--dative-- charge/VB
$/$ <--nmod-- 69/CD
69/CD <--dobj-- charge/VB
for/IN <--prep-- charge/VB
3/CD <--nummod-- pills/NNS
pills/NNS <--pobj-- for/IN
./. <--punct-- had/VBD
I/PRP <--nsubj-- checked/VBD
checked/VBD <--ROOT-- checked/VBD
online/RP <--advmod-- checked/VBD
the/DT <--det-- pills/NNS
pills/NNS <--nsubjpass-- had/VBN
can/MD <--aux-- had/VBN
be/VB <--auxpass-- had/VBN
had/VBN <--ccomp-- checked/VBD
for/IN <--prep-- had/VBN
19/CD <--nummod-- cents/NNS
cents/NNS <--pobj-- f

In [26]:
for token in doc:
    print("{0}:{1}".format(
        token.text, token.tag_))

Total:JJ
bill:NN
for:IN
this:DT
horrible:JJ
service:NN
?:.
Over:IN
$:$
8Gs:CD
.:.
These:DT
crooks:NNS
actually:RB
had:VBD
the:DT
nerve:NN
to:TO
charge:VB
us:PRP
$:$
69:CD
for:IN
3:CD
pills:NNS
.:.
I:PRP
checked:VBD
online:RP
the:DT
pills:NNS
can:MD
be:VB
had:VBN
for:IN
19:CD
cents:NNS
EACH:NNP
!:.
Avoid:VB
Hospital:NNP
ERs:NNP
at:IN
all:DT
costs:NNS
.:.


In [61]:
displacy.render(doc, style='dep', jupyter=True, options={'distance': 90})